# CS61A, Spring 2025 Prof Denero 
## String Representation, Interpolation, Polymorphic Functions, Special Method Names
### Sean Villegas

Videos:
- [Lectures](https://www.youtube.com/watch?v=DUSSKCjzAgA&list=PL6BsET-8jgYW8tpgR0-wtQEA2VyOyNXFB)


#### String Representations

- str is made to be understood by humans
- repr is made to be understood by python interpreter 
    - they can be the same thing sometimes 


**repr** for an object
- result of calling repr on a value is like interacting with python interpreter 
- calling repr can evaluate like thjis for most object types:
```python
eval(repr(object)) == object
```
When it evaluates different:
```python
repr(min)
'<built-in function min>'
```

**str** for an object
```python
>>> from fractions import Fraction
>>> half = Fraction(1, 2)
>>> half
Fraction(1, 2)
>>> repr(half)
'Fraction(1, 2)'
>>> print(half)
1/2
>>> str(half)
'1/2'
>>> eval(repr(half))
Fraction(1, 2)
>>> eval(str(half))
0.5 
```

_Looking deeper into string representation for interpreter_ 
```python
>>> s = "hello ambatakum"
>>> s

>>> print(repr(s))
'hello ambatakum'

>>> print(s)
hello ambatakum

>>> print(str(s))
hello ambatakum

>>> str(s) # same as repr 
'hello ambatakum'

>>> repr(s)
"'hello ambatakum'"

>>> eval(repr(s)) # same as string and print
'hello ambatakum'

>>> repr(repr(repr(s))) # python lets us know what quote ends the string of quotes within quotes
'\'"\\\'hello ambatakum\\\'"\''

>>> eval(eval(eval(repr(repr(repr(s))))))
'hello ambatakum'

>>> eval(s)
NameError: name is not defined 

>>> eval(repr(s))
'hello ambatakum'
```

#### String Interpolation

- exists in many different programming languages 
- f strings allow you to take expressions (that become sub expressions) within a string literal
    - sub expressions are evaluate in the current environment 
- those expressions are evaluated and put into the string literal, opposed to concatenating the str(object) with strings 

```python
>>> f'2 + 2 = 2 + 2'
'2 + 2 = 2 + 2'

>>> f'2 + 2 = {2 + 2}'
'2 + 2 = 4' 

>>> '2 + 2 = {2 + 2}' 
'2 + 2 = {2 + 2}'

>>> f'2 + 2 = {abs(2+2)}'
'2 + 2 = 4' 

## can use any legal expression with curly braces for evaluation of expression -> sub expression
>>> f'2+2={(lambda x: x + x)(2)}
'2 + 2 = 4'

>>> from fractions import Fraction
>>> half = Fraction(1, 2)
>>> half
Fraction(1, 2)
>>> print(half)
1/2
>>> f'half of a half is {half * half}.'
'half of a half is 1/4.'

>>> f'half of a half is {repr(half * half)}'
'half of a half is Fraction(1, 4)'

## expressions are evaluated in order for list literals

>>> s = [6, 5, 4]
>>> f'because {s.pop()} {s.pop()} {s}.'
'because 4 5 [6].'
```



#### Polymorphic Functions
- a func that applies to many forms of data (poly and morph definitions)
- str and repr are polymorphic, (applied to an object) 
- Note: looks like this is prepping for building scheme 
- A **zero-argument** method in Python is a function defined within a class that does not accept any explicit arguments other than the implicit self reference. This self parameter refers to the instance of the class and is automatically passed when the method is called.
    - `str invokes zero-arg method __str__ on argument`
    - `repr invokes zero-arg method __repr__ on argument`
- repr behavior underhood is complicated, an instance attribute called `__rpr__` is ignored, only class attributes are found
- str is also complicated, instance `__str__` is skipped, if no `__str__` attribute is found, uses repr string

Implementing this behavior: 
```python
def repr(x)
## skips instance attribute, looks up type of argument, that gets class, thus asking for the __rpr__ attribute of the class attribute. repr isnt bound when you call the method, so you have to pass in the variable to __rpr__ as well 
    return type(x).__repr__(x)
```

**Main takeaway: Interfaces in OOP**
Example real world:
- Message Passing Interface (MPI) for Python (mpi4py) is a library that enables Python programs to run in parallel across multiple processors. It is based on the MPI standard, a widely used communication protocol for parallel computing. MPI allows different processes to communicate with each other by sending and receiving messages, making it suitable for distributed memory systems like clusters and supercomputers. With mpi4py, users can write parallel Python code that leverages the power of multiple processors to solve complex problems more efficiently. 


#### Special Method Names in Python

**Main Ideas**
- type dispatching: practice of executing different code paths based on the type of the input
- type cohesion: The degree to which the elements inside a module or class belong together; high cohesion means all parts contribute to a single, well-defined purpose.


In [ ]:
class Calculator:
    def __init__(self, value=0):
        self.value = value

    # Special method for string representation
    def __str__(self):
        return f"Calculator(value={self.value})"

    # Example of type dispatching using __add__
    def __add__(self, other):
        if isinstance(other, (int, float)):
            return Calculator(self.value + other)
        elif isinstance(other, Calculator):
            return Calculator(self.value + other.value)
        else:
            raise TypeError("Unsupported type for addition")

    # Example of type dispatching using __radd__ # if user were to call function on right side 
    def __radd__(self, other):
        return self.__add__(other)

    # Example of cohesive functionality: maintaining consistent calculator behavior
    def multiply(self, factor):
        if isinstance(factor, (int, float)):
            return Calculator(self.value * factor)
        else:
            raise TypeError("Factor must be a number")

# Example Usage
calc1 = Calculator(10)
calc2 = Calculator(5)
print(calc1 + calc2)      # Calculator(value=15)
print(calc1 + 3)          # Calculator(value=13)
print(3 + calc1)          # Calculator(value=13)
print(calc1.multiply(2))  # Calculator(value=20)

